<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-MCD/blob/main/07-SVD_Espacios_Latentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd

path = ...
df = pd.read_excel(path)
df.head()

In [ ]:
nombres = list(df.iloc[:,0].values)
movies = df.iloc[:,1:].columns.to_list()
print(nombres)
print(movies)

In [ ]:
df_no_names = df.drop(columns=['Unnamed: 0'])
df_no_names

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_aux = df_no_names.replace(np.nan,0)
plt.figure(figsize=(10, 6))
sns.heatmap(df_aux, cmap='coolwarm', linewidths=0.5,yticklabels=nombres)
plt.title('Matriz de calificaciones')
plt.xlabel('Películas')
plt.ylabel('Usuarios')
plt.show()

## Imputación

In [ ]:
#Utilizando simple imputer para los datos faltantes
from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean')
imputer = SimpleImputer(strategy='constant', fill_value=0)
df_imputed = pd.DataFrame(imputer.fit_transform(df_no_names), columns=df_no_names.columns)
df_imputed

## Singular Value Decomposition

In [ ]:
#Haciendo truncatedSVD con n_components < #peliculas
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=5)
X_transformed = svd.fit_transform(df_imputed)
print(f"Matriz individuos-componentes latentes: {X_transformed.shape}")

In [ ]:
#Matriz components_
components = svd.components_.T
print(f"Matriz componentes-peliculas: {components.shape}")

## Máxima afinidad

In [ ]:
from numpy import argmax

x_arg = argmax(X_transformed, axis=1)
print(x_arg.shape)
x_arg

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.heatmap(X_transformed, cmap='coolwarm', annot=True, fmt='.2f', linewidths=0.5)
plt.title('Matriz Individuos - Componentes Latentes')
plt.xlabel('Componentes')
plt.ylabel('Individuos')
plt.show()

In [ ]:
#aplicandosela a components
components_arg = argmax(components, axis=1)
components_arg

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.heatmap(components, cmap='coolwarm', annot=True, fmt='.2f', linewidths=0.5)
plt.title('Matriz Peliculas - Componentes Latentes')
plt.xlabel('Componentes')
plt.ylabel('Películas')
plt.show()

## Peliculas agrupadas por grupos

In [ ]:
for j in range(X_transformed.shape[1]):
    movie_idxs = np.where(components_arg==j)[0]
    print(f"Grupo {j+1}:")
    for idx in movie_idxs:
        print(f"\t{movies[idx]}")

## Embeddings

In [ ]:
individuos_embeddings = X_transformed.copy()
peliculas_embeddings = components.copy()

In [ ]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, metric='cosine')
nn.fit(individuos_embeddings)

for j,x in enumerate(nombres):
    distances, indices = nn.kneighbors([individuos_embeddings[j]])
    idx = indices[0][1]
    print(f"Más similar a {x}: {nombres[idx]}")

In [ ]:
nn = NearestNeighbors(n_neighbors=5, metric='cosine')
nn.fit(peliculas_embeddings)

for j,x in enumerate(movies):
    distances, indices = nn.kneighbors([peliculas_embeddings[j]])
    idx = indices[0][1]
    print(f"Más similar a {x}: {movies[idx]}")